In [10]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt

In [11]:
# Define fuzzy variables
# Input variables
temperature = ctrl.Antecedent(np.arange(16, 31, 1), 'temperature')  # Room temperature (16°C to 30°C)
comfort = ctrl.Antecedent(np.arange(0, 11, 1), 'comfort')  # Comfort level (0 to 10)

# Output variable
fan_speed = ctrl.Consequent(np.arange(0, 101, 1), 'fan_speed')  # Fan speed percentage (0% to 100%)

In [12]:
# Define fuzzy membership functions
temperature['cold'] = fuzz.trimf(temperature.universe, [16, 16, 22])
temperature['comfortable'] = fuzz.trimf(temperature.universe, [20, 24, 28])
temperature['hot'] = fuzz.trimf(temperature.universe, [26, 30, 30])

comfort['low'] = fuzz.trimf(comfort.universe, [0, 0, 5])
comfort['medium'] = fuzz.trimf(comfort.universe, [3, 5, 7])
comfort['high'] = fuzz.trimf(comfort.universe, [6, 10, 10])

fan_speed['low'] = fuzz.trimf(fan_speed.universe, [0, 0, 50])
fan_speed['medium'] = fuzz.trimf(fan_speed.universe, [30, 50, 70])
fan_speed['high'] = fuzz.trimf(fan_speed.universe, [60, 100, 100])

In [13]:
# Define fuzzy rules
rules = [
    ctrl.Rule(temperature['cold'] & comfort['high'], fan_speed['low']),
    ctrl.Rule(temperature['cold'] & comfort['medium'], fan_speed['low']),
    ctrl.Rule(temperature['cold'] & comfort['low'], fan_speed['medium']),
    ctrl.Rule(temperature['comfortable'] & comfort['high'], fan_speed['low']),
    ctrl.Rule(temperature['comfortable'] & comfort['medium'], fan_speed['medium']),
    ctrl.Rule(temperature['comfortable'] & comfort['low'], fan_speed['high']),
    ctrl.Rule(temperature['hot'] & comfort['high'], fan_speed['medium']),
    ctrl.Rule(temperature['hot'] & comfort['medium'], fan_speed['high']),
    ctrl.Rule(temperature['hot'] & comfort['low'], fan_speed['high']),
]

fan_speed_control = ctrl.ControlSystem(rules)
fan_speed_sim = ctrl.ControlSystemSimulation(fan_speed_control)

In [14]:
# Function to display membership function curves
def show_curves():
    fig, axs = plt.subplots(3, 1, figsize=(8, 12))

    # Plot temperature membership functions
    axs[0].plot(temperature.universe, temperature['cold'].mf, label='Cold')
    axs[0].plot(temperature.universe, temperature['comfortable'].mf, label='Comfortable')
    axs[0].plot(temperature.universe, temperature['hot'].mf, label='Hot')
    axs[0].set_title('Temperature Membership Functions')
    axs[0].legend()

    # Plot comfort membership functions
    axs[1].plot(comfort.universe, comfort['low'].mf, label='Low')
    axs[1].plot(comfort.universe, comfort['medium'].mf, label='Medium')
    axs[1].plot(comfort.universe, comfort['high'].mf, label='High')
    axs[1].set_title('Comfort Membership Functions')
    axs[1].legend()

    # Plot fan speed membership functions
    axs[2].plot(fan_speed.universe, fan_speed['low'].mf, label='Low')
    axs[2].plot(fan_speed.universe, fan_speed['medium'].mf, label='Medium')
    axs[2].plot(fan_speed.universe, fan_speed['high'].mf, label='High')
    axs[2].set_title('Fan Speed Membership Functions')
    axs[2].legend()

    plt.tight_layout()
    plt.show()

In [15]:
# Tkinter GUI
def calculate_fan_speed():
    try:
        temp = float(temp_entry.get())
        comf = float(comfort_entry.get())
        fan_speed_sim.input['temperature'] = temp
        fan_speed_sim.input['comfort'] = comf
        fan_speed_sim.compute()
        result_label.config(text=f"Fan Speed: {fan_speed_sim.output['fan_speed']:.2f}%")
    except Exception as e:
        result_label.config(text="Error: Invalid input!")

root = tk.Tk()
root.title("Fuzzy Fan Speed Control")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

In [16]:
# Input fields
ttk.Label(frame, text="Temperature (16-30°C):").grid(row=0, column=0, sticky=tk.W)
temp_entry = ttk.Entry(frame, width=10)
temp_entry.grid(row=0, column=1)

ttk.Label(frame, text="Comfort Level (0-10):").grid(row=1, column=0, sticky=tk.W)
comfort_entry = ttk.Entry(frame, width=10)
comfort_entry.grid(row=1, column=1)

In [17]:
# Buttons
ttk.Button(frame, text="Calculate", command=calculate_fan_speed).grid(row=2, column=0, columnspan=2, pady=10)
ttk.Button(frame, text="Show Curves", command=show_curves).grid(row=3, column=0, columnspan=2, pady=10)


In [18]:

# Result display
result_label = ttk.Label(frame, text="Fan Speed: ")
result_label.grid(row=4, column=0, columnspan=2)

root.mainloop()